# Sección 1: Estudio Detallado de K-Nearest Neighbors (KNN)

## 1. ¿Qué es K-Nearest Neighbors (KNN)?
    
  K-Nearest Neighbors (KNN) o K-Vecinos más Cercanos es uno de los algoritmos más simples e intuitivos de **Aprendizaje Supervisado** (Supervised Learning). Se le conoce como un algoritmo \"basado en instancias\" o \"perezoso\" (*lazy algorithm*).
    
  **¿Por qué \"perezoso\"?:** A diferencia de otros algoritmos (como la Regresión Logística o las SVM) que construyen un modelo explícito durante la fase de \"entrenamiento\", KNN no \"aprende\" nada. Simplemente **memoriza todo el conjunto de datos de entrenamiento**.
  
  El verdadero \"trabajo\" ocurre durante la fase de **predicción** o **inferencia**.

  KNN puede usarse tanto para **Clasificación** como para **Regresión**.
  
  * **Clasificación:** Asigna la clase más común entre sus K vecinos más cercanos.
  * **Regresión:** Predice el valor promedio de los K vecinos más cercanos.

## 2. ¿Cómo Funciona el Algoritmo? (Clasificación)

  Imaginemos que tenemos un nuevo punto de datos (un punto \"incógnita\") al que queremos asignarle una clase.
    
  1.  **Elegir un valor para K:** K es el número de \"vecinos\" que el algoritmo considerará. Es un hiperparámetro clave. (Ej. K=5).
  2.  **Calcular Distancias:** Calcula la distancia entre el punto incógnita y **todos** los puntos del conjunto de entrenamiento. La métrica de distancia más común es la **Distancia Euclidiana**.
  3.  **Encontrar los K Vecinos:** Identifica los K puntos de entrenamiento que tienen las distancias más cortas al punto incógnita.
  4.  **Votar por la Clase:** Mira las etiquetas (clases) de esos K vecinos. La clase que más se repite (la \"moda\") se asigna como la predicción para el punto incógnita.

## 3. El Hiperparámetro Clave: K

  La elección de K es crucial y afecta directamente el **balance entre sesgo y varianza** (*bias-variance tradeoff*).

  * **K pequeño (ej. K=1):**
      * **Bajo Sesgo (Low Bias):** El modelo es muy flexible y se adapta mucho a los datos de entrenamiento.
      * **Alta Varianza (High Variance):** Es muy sensible al ruido y a los *outliers*. Un solo punto atípico puede cambiar una predicción. Es propenso al **sobreajuste (Overfitting)**.
  
  * **K grande (ej. K=N, donde N es total de puntos):**
       * **Alto Sesgo (High Bias):** El modelo es muy simple (siempre predecirá la clase mayoritaria de todo el dataset).
       * **Baja Varianza (Low Variance):** Es muy estable, pero pierde los patrones locales. Es propenso al **subajuste (Underfitting)**.

  **¿Cómo elegir K?**
  Generalmente se usa **Validación Cruzada (Cross-Validation)**. Se prueba un rango de valores de K (usualmente impares, para evitar empates) y se elige el que dé la mejor métrica de evaluación (ej. accuracy, F1-score) en el conjunto de validación. Un método común es el \"Método del Codo\" (Elbow Method).

## 4. Métricas de Distancia
    
  KNN depende fundamentalmente de la "distancia". La elección de la métrica puede cambiar los resultados.
  
  Dados dos puntos $p$ y $q$ en un espacio de $n$ dimensiones ($p = (p_1, p_2, ..., p_n)$ y $q = (q_1, q_2, ..., q_n)$):
    
  ### a) Distancia Euclidiana ($L_2$)
  "Es la distancia \"en línea recta\" que todos conocemos. Es la más usada.
    "$$ d(p, q) = \\sqrt{\\sum_{i=1}^{n} (p_i - q_i)^2} $$
  
  ### b) Distancia de Manhattan ($L_1$)
  Es la suma de las distancias absolutas de las coordenadas. Útil en espacios de alta dimensionalidad o cuando las \"calles\" (features) son ortogonales.\n",
    "$$ d(p, q) = \\sum_{i=1}^{n} |p_i - q_i| $$
  
  ### c) Distancia de Minkowski ($L_p$)
  Es una generalización de las dos anteriores. El parámetro $p$ define la métrica:
    "$$ d(p, q) = \\left( \\sum_{i=1}^{n} |p_i - q_i|^p \\right)^{1/p} $$\n",
  
  * Si $p=1$, es la Distancia de Manhattan.
  * Si $p=2$, es la Distancia Euclidiana.

## 5. ¡IMPORTANTE! La Necesidad de Escalar los Datos
    
  Este es uno de los puntos más críticos de KNN. **El algoritmo es extremadamente sensible a la escala de las características (features).**
    
  Imagina que tienes un dataset para predecir si un cliente comprará, con dos features:
  1.  `edad` (rango: 18 - 80)
  2.  `ingreso_anual` (rango: 20,000 - 150,000)
    
  Si usamos la distancia euclidiana, la diferencia en `ingreso_anual` (que está en decenas de miles) **dominará completamente** el cálculo de la distancia, haciendo que la `edad` sea prácticamente irrelevante.
    
  **Solución:** Siempre se deben escalar los datos antes de usar KNN. Métodos comunes:
  * **Estandarización (StandardScaler):** Transforma los datos para que tengan media 0 y desviación estándar 1. (Recomendado si hay outliers).
  * **Normalización (MinMaxScaler):** Escala los datos a un rango fijo, comúnmente [0, 1].

## 6. Implementación (Código)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Clases de Scikit-learn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler\n",
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.inspection import DecisionBoundaryDisplay

### 6.1. Implementación de KNN "Desde Cero"

In [ ]:
# Función para calcular la distancia euclidiana
def euclidean_distance(x1, x2):
      return np.sqrt(np.sum((x1 - x2)**2))

class KNN_from_scratch:
  def __init__(self, k=3):
      self.k = k


  def fit(self, X, y):\n",
     # KNN es \"perezoso\", solo memoriza los datos
     self.X_train = X
     self.y_train = y

  def predict(self, X_test):
      # Hacemos predicciones para cada punto en X_test
      y_pred = [self._predict_point(x) for x in X_test]
      return np.array(y_pred)

  def _predict_point(self, x):
     # 1. Calcular distancias a todos los puntos de entrenamiento
    distances = [euclidean_distance(x, x_train) for x_train in self.X_train]

    # 2. Obtener los índices de los K vecinos más cercanos
    # np.argsort() devuelve los índices que ordenarían el array
    k_nearest_indices = np.argsort(distances)[:self.k]

    # 3. Obtener las etiquetas (clases) de esos K vecinos
    k_nearest_labels = [self.y_train[i] for i in k_nearest_indices]

    # 4. Votar: Devolver la clase más común (la moda)
    # Counter(...).most_common(1) devuelve [(etiqueta, conteo)]
    most_common = Counter(k_nearest_labels).most_common(1)
    return most_common[0][0]

### 6.2. Implementación con `scikit-learn` (Uso Real)

Ahora usemos la implementación optimizada de `scikit-learn` con el famoso dataset Iris.

In [ ]:
# 1. Cargar Datos\n",
iris = load_iris()
X = iris.data
y = iris.target

# 2. Dividir en Entrenamiento y Prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 3. Escalar los Datos (¡Muy importante!)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 4. Encontrar el mejor K (Método del Codo)
k_range = range(1, 21)
accuracy_scores = []

for k in k_range:
    knn = KNeighborsClassifier(n_neighbors=k)
    # Usamos validación cruzada (5-fold) en los datos de entrenamiento
    scores = cross_val_score(knn, X_train_scaled, y_train, cv=5, scoring='accuracy')
    accuracy_scores.append(scores.mean())

    # Graficar los resultados
    plt.figure(figsize=(10, 6))
    plt.plot(k_range, accuracy_scores, marker='o', linestyle='dashed')
    plt.title('Precisión vs. Valor de K (Método del Codo)')
    plt.xlabel('Valor de K')
    plt.ylabel('Precisión (Validación Cruzada)')
    plt.xticks(k_range)
    plt.grid(True)
    plt.show()

    # Elegir el mejor K (el que maximiza la precisión)
    best_k = k_range[np.argmax(accuracy_scores)]
    print(f\"El mejor valor de K encontrado es:" {best_k})

# 5. Entrenar y Evaluar el Modelo Final con el mejor K

In [ ]:
    print(f\"Entrenando modelo final con K={best_k}...")

    # Instanciamos el clasificador de Scikit-learn\n",
    knn_final = KNeighborsClassifier(n_neighbors=best_k)

    # \"Entrenamos\" (memorizamos) los datos escalados
    knn_final.fit(X_train_scaled, y_train)

    # Hacemos predicciones en el conjunto de prueba
    y_pred = knn_final.predict(X_test_scaled)

# 6. Evaluar el rendimiento

In [ ]:
    accuracy = accuracy_score(y_test, y_pred)
    print(f\"\\nPrecisión (Accuracy) en el conjunto de prueba:" {accuracy:.4f})

    print(\"\\nReporte de Clasificación:")
    print(classification_report(y_test, y_pred, target_names=iris.target_names))

    print(\"\\nMatriz de Confusión:")
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=iris.target_names, yticklabels=iris.target_names)
    plt.xlabel('Predicción')
    plt.ylabel('Valor Real')
    plt.show()

## 7. Ventajas y Desventajas de KNN
    
  ### Ventajas
  1.  **Simple e Intuitivo:** Fácil de entender y explicar.
  2.  **Sin Fase de Entrenamiento:** Como es \"perezoso\", la \"capacitación\" es instantánea (solo almacena datos).
  3.  **Flexible:** Se adapta fácilmente a nuevos datos de entrenamiento (solo hay que agregarlos al dataset memorizado).
  4.  **No Paramétrico:** No hace suposiciones sobre la distribución de los datos (ej. no asume que son linealmente separables).
  5.  **Bueno para Tareas No Lineales:** Puede capturar fronteras de decisión complejas.
  
  ### Desventajas
  1.  **Costo Computacional en Predicción:** ¡Es muy lento para predecir! Debe calcular la distancia a *todos* los puntos de entrenamiento por cada nueva predicción. (Complejidad $O(N*D)$ donde N=muestras, D=features).

  2.  **La Maldición de la Dimensionalidad (*Curse of Dimensionality*):**
      * En espacios con muchas dimensiones (muchas features), todos los puntos tienden a estar \"lejos\" entre sí.
      * El concepto de \"vecino cercano\" pierde sentido.
      * El rendimiento se degrada rápidamente con $D$ alto.
      
  3.  **Sensible a Features Irrelevantes:** Si hay muchas features que no aportan información (ruido), estas pueden opacar a las features útiles en el cálculo de la distancia.
  4.  **Requiere Escalamiento de Datos:** Como vimos, es obligatorio.
  5.  **Necesita Muchos Datos:** Requiere una buena densidad de datos para que el concepto de "vecino" sea significativo. Consume mucha memoria (almacena todo $X_{train}$).

# Sección 2: Aplicación Hipotética y Modificación del Algoritmo
    
  Basándonos en el estudio anterior, vamos a plantear un escenario donde el KNN estándar falla y cómo podemos adaptarlo.


## 1. El Problema: Datos Desbalanceados
    
  **Aplicación Hipotética:** Detección de Fraude en Transacciones.
    
  Tenemos un dataset de transacciones bancarias. La gran mayoría (ej. 99.5%) son legítimas (Clase 0) y una pequeñísima minoría (0.5%) son fraudulentas (Clase 1).
    
  **¿Por qué falla el KNN estándar (voto por mayoría)?**
    
  Imagina que usamos K=10 para clasificar una nueva transacción (punto $X_{nuevo}$).
  Debido a que la Clase 0 (legítima) es abrumadoramente dominante, es **extremadamente probable** que la mayoría (quizás 8, 9 o incluso los 10) de los vecinos más cercanos pertenezcan a la Clase 0, simplemente porque hay muchos más.
  
  Puede que el vecino *más cercano* (el #1) sea de Clase 1 (fraude), pero si los otros 9 son de Clase 0, el voto por mayoría predecirá Clase 0. El modelo estará fuertemente sesgado hacia la clase mayoritaria.
  
## 2. La Modificación: KNN Ponderado (Weighted KNN)

  **Idea Central:** No todos los vecinos deben tener el mismo peso en la votación. **Un vecino que está más cerca debe tener más influencia que un vecino que está más lejos.**
  
  **La Adaptación (Cómo funciona):**

  1.  Encuentra los K vecinos más cercanos y sus distancias (igual que antes).
  2.  Calcula un **peso (weight)** para cada vecino. Una función común es la inversa de la distancia:
      $$ weight_i = \\frac{1}{distance_i + \\epsilon} $$
      (Se añade un épsilon, un número muy pequeño, para evitar la división por cero si la distancia es 0).
  3.  **Votación Ponderada:** En lugar de que cada vecino dé 1 voto, cada vecino da un voto igual a su `peso`.
      * Se suman los pesos de todos los vecinos que pertenecen a la Clase 0.
      * Se suman los pesos de todos los vecinos que pertenecen a la Clase 1.
  4.  **Predicción:** La clase que acumule el mayor peso total gana la votación.
    
  **¿Por qué esto soluciona el problema?**
  Volviendo al ejemplo: Si el vecino #1 (Clase 1) está a una distancia de 0.1, y los otros 9 vecinos (Clase 0) están a distancias de 0.8, 0.9, 1.0, etc...

  El peso del vecino de Clase 1 (ej. $1/0.1 = 10$) será mucho mayor que la suma de los pesos de los vecinos de Clase 0 (ej. $1/0.8 + 1/0.9 + ... < 10$).
    
  El **KNN Ponderado** le da más importancia a la *proximidad* que al simple *conteo*.

## 3. Implementación de KNN Ponderado
    
  ### 3.1. Modificando nuestra clase "Desde Cero"

In [ ]:
class Weighted_KNN_from_scratch:
    def __init__(self, k=3):
        self.k = k
        self.epsilon = 1e-6 # Para evitar división por cero

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X_test):
        y_pred = [self._predict_point(x) for x in X_test]
        return np.array(y_pred)

    def _predict_point(self, x):
        # 1. Calcular distancias
        distances = [euclidean_distance(x, x_train) for x_train in self.X_train]

        # 2. Obtener índices y distancias de los K vecinos
        k_nearest_indices = np.argsort(distances)[:self.k]
        k_nearest_distances = [distances[i] for i in k_nearest_indices]
        k_nearest_labels = [self.y_train[i] for i in k_nearest_indices]

        # 3. Calcular Pesos (¡Aquí está la modificación!)
        weights = [1 / (d + self.epsilon) for d in k_nearest_distances]

        # 4. Votación Ponderada
        class_weights = {}
        for i in range(self.k):
            label = k_nearest_labels[i]
            weight = weights[i]

            if label not in class_weights:
                class_weights[label] = 0
            class_weights[label] += weight

        # 5. Devolver la clase con el mayor peso acumulado
        # (max(..., key=...) devuelve la clave del diccionario con el valor máximo)
        return max(class_weights, key=class_weights.get)

### 3.2. Comparación en `scikit-learn`
    
  Afortunadamente, `scikit-learn` ya tiene esta funcionalidad implementada con el hiperparámetro `weights`.
    
  * `weights='uniform'` (Default): Voto por mayoría estándar. Todos los vecinos pesan 1.
  * `weights='distance'` (Modificado): Voto ponderado por la inversa de la distancia (justo lo que implementamos).
  
  Vamos a crear un dataset sintético **desbalanceado** y comparar el rendimiento.

In [1]:
from sklearn.datasets import make_classification

# Crear un dataset con 2 features, 1000 muestras, 95% de Clase 0 y 5% de Clase 1
X_imb, y_imb = make_classification(n_samples=1000,
                                 n_features=2,
                                 n_informative=2,
                                 n_redundant=0,
                                 n_clusters_per_class=1,
                                 weights=[0.95, 0.05], # <-- ¡Desbalanceado!,
                                 flip_y=0,
                                 random_state=42)

print(f\"Forma de X: " {X_imb.shape})
print(f\"Conteo de clases: " {Counter(y_imb)})

# Graficar los datos desbalanceados
sns.scatterplot(x=X_imb[:, 0], y=X_imb[:, 1], hue=y_imb, style=y_imb, palette={0: 'blue', 1: 'red'})
plt.title('Dataset Sintético Desbalanceado (0=Azul, 1=Rojo)')
plt.show()

SyntaxError: unexpected character after line continuation character (ipython-input-1648855124.py, line 13)

In [ ]:
# Dividir y escalar los datos desbalanceados
X_train_imb, X_test_imb, y_train_imb, y_test_imb = train_test_split(X_imb, y_imb, test_size=0.3, random_state=42, stratify=y_imb)

scaler_imb = StandardScaler()
X_train_imb_scaled = scaler_imb.fit_transform(X_train_imb)
X_test_imb_scaled = scaler_imb.transform(X_test_imb)

# K fijo para la comparación (ej. K=7)
K_COMPARE = 7

# Modelo 1: KNN Estándar (weights='uniform')
knn_uniform = KNeighborsClassifier(n_neighbors=K_COMPARE, weights='uniform')
knn_uniform.fit(X_train_imb_scaled, y_train_imb)
y_pred_uniform = knn_uniform.predict(X_test_imb_scaled)

# Modelo 2: KNN Ponderado (weights='distance')
knn_distance = KNeighborsClassifier(n_neighbors=K_COMPARE, weights='distance')
knn_distance.fit(X_train_imb_scaled, y_train_imb)
y_pred_distance = knn_distance.predict(X_test_imb_scaled)

# --- Comparar Resultados ---
# En datos desbalanceados, 'accuracy' es engañoso. Usamos 'classification_report' (F1-score, Recall)

print(\"--- RESULTADOS KNN ESTÁNDAR (weights='uniform') ---\")
print(classification_report(y_test_imb, y_pred_uniform, target_names=['Clase 0 (Legítima)', 'Clase 1 (Fraude)']_)

print(\"\\n--- RESULTADOS KNN PONDERADO ---\ " (weights='distance'))
print(classification_report(y_test_imb, y_pred_distance, target_names=['Clase 0 (Legítima)', 'Clase 1 (Fraude)']))

### 3.3. Análisis de la Comparación
    
  Al ejecutar el código anterior, deberías notar lo siguiente (los números exactos pueden variar):
    
  1.  **KNN Estándar (`uniform`):**
      * Tendrá un `recall` (Sensibilidad) muy bajo para la Clase 1 (Fraude). Esto significa que falla en identificar la mayoría de los casos de fraude. Simplemente predice Clase 0 casi siempre.
    
  2.  **KNN Ponderado (`distance`):**
      * Tendrá un `recall` para la Clase 1 significativamente mejor.
      * Al dar más peso a los vecinos *muy* cercanos, es capaz de identificar correctamente instancias de la clase minoritaria (fraude) aunque esté rodeada por puntos de la clase mayoritaria.
    
  **Conclusión de la Adaptación:**
  El estudio detallado de la Sección 1 nos mostró que KNN se basa en "distancia" y "votación". La aplicación hipotética (datos desbalanceados) reveló una debilidad en la "votación" estándar (voto por mayoría).
    
  La modificación (KNN Ponderado) cambia el mecanismo de "votación" para incorporar la "distancia", creando un modelo mucho más robusto para este caso de uso específico.